In [ ]:
import polars as pl
import polars_ds as pld
import numpy as np

In [ ]:
import polars_ds as pld
size = 5000
df = pl.DataFrame({
    "id": range(size),
    "val1": np.random.random(size=size), 
    "val2": np.random.random(size=size), 
    "val3": np.random.random(size=size),
    "r": np.random.random(size=size),
    "rh": np.random.random(size=size)*10,
}).with_columns(
    pl.col("id").cast(pl.UInt64)
)

In [ ]:
test = df.select(
    pl.col("val1").num.rfft().alias("arr"),
    pl.col("val2").num.rfft().arr.to_list().alias("list"),
) 
test.head()

In [ ]:
%%timeit
test.select(
    pl.col("arr").arr.first().pow(2) + pl.col("arr").arr.last().pow(2)
)

In [ ]:
%%timeit
test.select(
    pl.col("list").list.eval(pl.element().dot(pl.element()))
)

In [ ]:
df.select(
    pl.concat_list(pl.col("val1"), pl.col("val2")).alias("list")
).with_columns(
    pl.col("list").list.to_array(2)
)

In [ ]:

df = df.with_columns(
    pl.col("id").num.query_radius_ptwise(
        pl.col("val1"), pl.col("val2"), pl.col("val3"), # Columns used as the coordinates in n-d space
        r = 0.05, 
        dist = "l2", # actually this is squared l2
        parallel = True,
    ).alias("best friends")
) 
df

In [ ]:
df.select(
    pl.col("best friends").list.len()
)

In [ ]:
df.select(
    pl.col("best friends").graph.shortest_path_const_cost(19, parallel=True)
)

In [ ]:
df.with_columns(
    pl.col("a").stats.rand_str(min_size=4,max_size=6).alias("s1"),
    # pl.col("a").stats.rand_str(min_size=4,max_size=6).alias("s2"),
) 

In [ ]:
df.with_columns(
    pl.col("s1").str2.hamming(pl.col("s2"), pad = True).alias("hamming pad"),
    pl.col("s1").str2.hamming(pl.col("s2"), pad = False).alias("hamming no pad"),
)

In [ ]:
df.with_columns(
    pl.col("s1").str2.hamming(pl.col("s2")).alias("hamming")
).filter(
    pl.col("s1").str2.hamming_filter(pl.col("s2"), bound = 2)
)

In [ ]:
%%timeit
df.select(
    pl.col("s1").str2.levenshtein(pl.col("s2")) < 3
) # 3.25ms

In [ ]:
%%timeit
df.select(
    pl.col("s1").str2.levenshtein_filter(pl.col("s2"), 3)
) # 2.43ms